# Data Preparation Notebook

This represents a data engineering task

In [ ]:
dbutils.widgets.text("environment", "dev")

In [ ]:
from pyspark.sql import functions as F
from datetime import datetime, timedelta

In [ ]:
%sql

CREATE CATALOG IF NOT EXISTS brian_ml_${environment};
USE CATALOG brian_ml_${environment};
CREATE SCHEMA IF NOT EXISTS warehouse;

In [ ]:
raw_data = spark.read.format("delta").load("/databricks-datasets/nyctaxi-with-zipcodes/subsampled")
display(raw_data)

We want to select just 1 days worth of data so that we can run a regular timed job and populate the table bit by bit

In [ ]:
min_datetime = raw_data \
    .select('tpep_pickup_datetime') \
    .agg(
        F.min(F.col('tpep_pickup_datetime')).alias('first_date')
    )

min_date = min_datetime.collect()[0].first_date
next_day = min_date + timedelta(days=1)
next_day_midnight = next_day.replace(hour=0,minute=0,second=0)
next_day_midnight

In [ ]:
# filter the raw dataframe and create the table
filtered_df = raw_data.filter(
    F.col('tpep_pickup_datetime') <= next_day_midnight
)
print(f'collected {filtered_df.count()} records')

In [ ]:
filtered_df.write.mode('overwrite').saveAsTable(f'warehouse.raw_data')

In [ ]:
%sql

ALTER TABLE warehouse.raw_data SET TBLPROPERTIES (delta.enableChangeDataFeed = true)